In [22]:
from cobra.io import read_sbml_model
import pandas as pd

ceres_stats_df = pd.read_parquet('../data/processed/ceres_stats_df.parquet')
sbml_fname = '../models/Recon3DModel_301_no_isoforms_gene_symbols_LT15_trimed.xml'
model = read_sbml_model(sbml_fname)
model.objective = "biomass_reaction"
model.solver = 'glpk'
solution = model.optimize()
objective_value = solution.objective_value
insilico_lethals, insilico_non_lethals = knockouts(model)
lethal_ceres_df = ceres_stats_df.loc[ceres_stats_df.index.intersection(insilico_lethals)]
non_lethal_ceres_df = ceres_stats_df.loc[ceres_stats_df.index.intersection(insilico_non_lethals)]
all_genes = [g.id for g in model.genes]
bestmodel_df = pd.DataFrame(index = all_genes)
essential_genes_dict = {g:('1' if g in insilico_lethals else '0') for g in all_genes}
bestmodel_df['WT'] = pd.Series(essential_genes_dict)

'' is not a valid SBML 'SId'.


in-silico lethals: 28
in-silico non lethals: 1855


In [38]:
bestmodel_df = bestmodel_df.astype(int)

In [10]:
def knockouts(model):
    # Import the function single_gene_deletion
    from cobra.flux_analysis import single_gene_deletion

    # Then get the list of all the genes
    all_genes = [g.id for g in model.genes]

    # Running in silico (takes a while)
    knockout = single_gene_deletion(model, gene_list=all_genes)

    # This is a fix to get the gene's id as the index
    knockout['ids'] = [list(i)[0] for i in knockout.ids]
    knockout = knockout.set_index('ids')
    # We define a threshold to define whether the reduction on the biomass flux is considered lethal.
    objective_value = solution.objective_value
    threshold = objective_value*0.1

    # Use this threshold to find which set of genes' knock out reduce the predicted growth below the threshold.
    insilico_lethals = set(knockout.index[knockout.growth< threshold])
    # The set of non-essential genes are the genes with a growth value above the threshold.
    insilico_non_lethals = set(knockout.index[knockout.growth > threshold])

    print("in-silico lethals:", len(insilico_lethals))
    print("in-silico non lethals:", len(insilico_non_lethals))
    return insilico_lethals, insilico_non_lethals

In [63]:
from cobra.io import read_sbml_model, validate_sbml_model
from os import listdir
import scipy.stats
import pandas as pd

path = '../models/ga_models/'

top_essentials_df = pd.DataFrame()
ceres_stats_df = pd.read_parquet('../data/processed/ceres_stats_df.parquet')

pval_dict = {}

for file in listdir(path):
    sbml_fname = str(path+file)
#sbml_fname = '../models/Recon3DModel_301_no_isoforms_gene_symbols_LT15_trimed.xml'
# Read the model
    print('Reading model:', file)
    model = read_sbml_model(sbml_fname)
    model.objective = "biomass_reaction"
    model.solver = 'glpk'
    solution = model.optimize()
    objective_value = solution.objective_value
    insilico_lethals, insilico_non_lethals = knockouts(model)
    lethal_ceres_df = ceres_stats_df.loc[ceres_stats_df.index.intersection(insilico_lethals)]
    non_lethal_ceres_df = ceres_stats_df.loc[ceres_stats_df.index.intersection(insilico_non_lethals)]
    result = scipy.stats.mannwhitneyu(lethal_ceres_df['mean_ceres'],non_lethal_ceres_df['mean_ceres'])
    statistic = result.statistic
    pval = result.pvalue
    print(pval)
    aux_colname = file.split('.')[0]
    colname = aux_colname.split('model_')[1]
    essential_genes_dict = {}
    all_genes = [g.id for g in model.genes]
    essential_genes_dict = {g:('1' if g in insilico_lethals else '0') for g in all_genes}
    pval_dict[colname] = pval
    bestmodel_df[colname] = pd.Series(essential_genes_dict)

Reading model: Recon3D_LT15_trimed_corda_model_10_29_81.xml


'' is not a valid SBML 'SId'.


in-silico lethals: 61
in-silico non lethals: 1353
9.592041241540767e-17
           WT 10_29_81  10_56_82  11_39_76  11_47_91  11_59_83  11_69_93  \
unknown_0   0        0       0.0         0         0         0         0   
ACOT8       0        0       0.0         0         0         0         0   
GNPDA1      0      NaN       NaN         0         0         0         0   
PLA2G4B     0        0       0.0         0         0         0         0   
GNE         0        0       0.0         0         0         0         0   

           11_73_92  12_34_75  12_45_81  
unknown_0         0         0         0  
ACOT8             0         0         0  
GNPDA1            0         0         0  
PLA2G4B           0         0         0  
GNE               0         0         0  
Reading model: Recon3D_LT15_trimed_corda_model_10_56_82.xml


'' is not a valid SBML 'SId'.


in-silico lethals: 61
in-silico non lethals: 1323
9.530306180556171e-17
           WT 10_29_81 10_56_82  11_39_76  11_47_91  11_59_83  11_69_93  \
unknown_0   0        0        0         0         0         0         0   
ACOT8       0        0        0         0         0         0         0   
GNPDA1      0      NaN      NaN         0         0         0         0   
PLA2G4B     0        0        0         0         0         0         0   
GNE         0        0        0         0         0         0         0   

           11_73_92  12_34_75  12_45_81  
unknown_0         0         0         0  
ACOT8             0         0         0  
GNPDA1            0         0         0  
PLA2G4B           0         0         0  
GNE               0         0         0  
Reading model: Recon3D_LT15_trimed_corda_model_11_39_76.xml


...

in-silico lethals: 61
in-silico non lethals: 1357
9.164020967616729e-17
           WT 10_29_81 10_56_82 11_39_76 11_47_91 11_59_83 11_69_93 11_73_92  \
unknown_0   0        0        0        0        0        0        0        0   
ACOT8       0        0        0        0        0        0        0        0   
GNPDA1      0      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
PLA2G4B     0        0        0        0      NaN        0      NaN      NaN   
GNE         0        0        0        0      NaN      NaN      NaN      NaN   

          12_34_75 12_45_81  ... 7_33_85 7_58_95 7_64_93 8_27_84 8_47_88  \
unknown_0        0        0  ...       0       0       0       0       0   
ACOT8            0        0  ...       0       0       0       0       0   
GNPDA1         NaN      NaN  ...     NaN     NaN     NaN     NaN     NaN   
PLA2G4B          0        0  ...       0     NaN     NaN       0       0   
GNE              0        0  ...       0       0       0       0   

'' is not a valid SBML 'SId'.


in-silico lethals: 67
in-silico non lethals: 1105
1.7736454077088923e-16
           WT 10_29_81 10_56_82 11_39_76 11_47_91 11_59_83 11_69_93 11_73_92  \
unknown_0   0        0        0        0        0        0        0        0   
ACOT8       0        0        0        0        0        0        0        0   
GNPDA1      0      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
PLA2G4B     0        0        0        0      NaN        0      NaN      NaN   
GNE         0        0        0        0      NaN      NaN      NaN      NaN   

          12_34_75 12_45_81  ... 7_58_95 7_64_93 8_27_84 8_47_88 8_60_95  \
unknown_0        0        0  ...       0       0       0       0       0   
ACOT8            0        0  ...       0       0       0       0       0   
GNPDA1         NaN      NaN  ...     NaN     NaN     NaN     NaN     NaN   
PLA2G4B          0        0  ...     NaN     NaN       0       0     NaN   
GNE              0        0  ...       0       0       0       0  

'' is not a valid SBML 'SId'.


in-silico lethals: 67
in-silico non lethals: 1146
9.845076904817639e-17
           WT 10_29_81 10_56_82 11_39_76 11_47_91 11_59_83 11_69_93 11_73_92  \
unknown_0   0        0        0        0        0        0        0        0   
ACOT8       0        0        0        0        0        0        0        0   
GNPDA1      0      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
PLA2G4B     0        0        0        0      NaN        0      NaN      NaN   
GNE         0        0        0        0      NaN      NaN      NaN      NaN   

          12_34_75 12_45_81  ... 7_64_93 8_27_84 8_47_88 8_60_95 8_61_80  \
unknown_0        0        0  ...       0       0       0       0       0   
ACOT8            0        0  ...       0       0       0       0       0   
GNPDA1         NaN      NaN  ...     NaN     NaN     NaN     NaN     NaN   
PLA2G4B          0        0  ...     NaN       0       0     NaN       0   
GNE              0        0  ...       0       0       0       0   

In [66]:
bestmodel_df_copy = bestmodel_df.copy(True)

In [85]:
bestmodel_df_copy = bestmodel_df_copy.fillna(0)
bestmodel_df_copy = bestmodel_df_copy.astype(float)
bestmodel_df_copy['freq'] = bestmodel_df_copy.apply(lambda row: sum(row)/87*100, axis=1)
bestmodel_df_copy

,WT,10_29_81,10_56_82,11_39_76,11_47_91,11_59_83,11_69_93,11_73_92,12_34_75,12_45_81,...,8_27_84,8_47_88,8_60_95,8_61_80,8_63_94,9_33_78,9_46_80,9_50_90,9_63_89,freq
unknown_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACOT8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GNPDA1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PLA2G4B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GNE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HS3ST2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HS3ST1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SLC23A2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SLC23A1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
bestmodel_df_copy[bestmodel_df_copy['freq'] < 100][bestmodel_df_copy['freq'] > 0].sort_values(by=['freq'], ascending = False)

<ipython-input-122-e2e7c22a9201>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bestmodel_df_copy[bestmodel_df_copy['freq'] < 100][bestmodel_df_copy['freq'] > 0].sort_values(by=['freq'], ascending = False)


,WT,10_29_81,10_56_82,11_39_76,11_47_91,11_59_83,11_69_93,11_73_92,12_34_75,12_45_81,...,8_27_84,8_47_88,8_60_95,8_61_80,8_63_94,9_33_78,9_46_80,9_50_90,9_63_89,freq
COL4A3BP,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575
COX8A,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575
UQCRB,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575
HSD17B7,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575
MT-CYB,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575
COX8C,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575
UQCRC1,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575
GUK1,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575
UQCR10,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575
UQCRC2,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575


In [117]:
bestmodel_df_copy.index

Index(['unknown_0', 'ACOT8', 'GNPDA1', 'PLA2G4B', 'GNE', 'PIGK', 'MTHFD2P1',
       'SDR42E2', 'AK4', 'SLC17A4',
       ...
       'XYLB', 'GFPT2', 'HS3ST4', 'HS3ST3B1', 'HS3ST3A1', 'HS3ST2', 'HS3ST1',
       'SLC23A2', 'SLC23A1', 'SLC12A6'],
      dtype='object', length=1883)

In [92]:
processed_path = '../data/processed'
bestmodel_df_copy.to_parquet(f'{processed_path}/bestmodel_table_biomass.parquet')

In [101]:
index = pval_dict.keys()
pval_df = pd.DataFrame(index = index)
pval_df['pval'] = pd.Series(pval_dict)
pval_df = pval_df.astype(float)
pval_df.to_parquet(f'{processed_path}/pvalues_df.parquet')

In [103]:
pval_df.sort_values(by=['pval'])

,pval
8_60_95,2.069302e-19
7_58_95,2.301672e-19
5_74_94,4.063113e-18
8_63_94,4.889656e-18
11_69_93,6.213030e-18
...,...
18_46_80,4.026225e-13
20_53_90,4.992073e-13
20_38_90,4.992073e-13
17_53_77,5.243072e-13


In [112]:
bestmodel_df_copy['8_60_95'].sum()

68.0

In [3]:
import pandas as pd
processed_path = '../data/processed'
bestmodel_df = pd.read_parquet(f'{processed_path}/bestmodel_table_biomass.parquet')

In [13]:
bestmodel_df[bestmodel_df['freq']<100].sort_values(by=['freq'], ascending=False)[:60]

,WT,10_29_81,10_56_82,11_39_76,11_47_91,11_59_83,11_69_93,11_73_92,12_34_75,12_45_81,...,8_27_84,8_47_88,8_60_95,8_61_80,8_63_94,9_33_78,9_46_80,9_50_90,9_63_89,freq
UQCRB,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575
SLC7A6,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575
COX8A,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575
COX7C,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575
COX7B,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575
COX7A2,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575
COX6C,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575
COX6B1,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575
COX6A2,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575
COX6A1,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.850575


In [8]:
bestmodel_df_copy[bestmodel_df_copy['freq'] < 100].sort_values(by=['freq'], ascending = False)[:40]

,WT,10_29_81,10_56_82,11_39_76,11_47_91,11_59_83,11_69_93,11_73_92,12_34_75,12_45_81,...,8_27_84,8_47_88,8_60_95,8_61_80,8_63_94,9_33_78,9_46_80,9_50_90,9_63_89,freq
CYP4F8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.824151
CYP51A1,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.824151
DHCR7,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.824151
DHCR24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.824151
TM7SF2,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,98.824151
SC5D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.824151
RRM2B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,93.882944
PNP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,93.882944
GPD1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,86.471132
PCYT2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,84.000528
